# Environemnt Setup and Prototype

## Sprint 1: Project Setup and Prototype

### 1.1 Data Basic Preporcessing

#### Import Libraries

In [20]:
# Import libraries
import pandas as pd
import os

#### Load Data
The CBIS-DDSM datset contains: 
- A metadata file in csv format
- Four descriptions files in csv format
- One directory with images stored in directories with a path based on the description of the case.

In [181]:
# Load data from metadata file
meta_path = 'CBIS-DDSM_Data/images_dataset/metadata.csv' 
metadata_df = pd.read_csv(meta_path)

# Load data from csv data files for training and testing sets
mass_train_orig = pd.read_csv("CBIS-DDSM_Data/mass_case_description_train_set.csv")
mass_test_orig = pd.read_csv("CBIS-DDSM_Data/mass_case_description_test_set.csv")
calc_train_orig = pd.read_csv("CBIS-DDSM_Data/calc_case_description_train_set.csv")
calc_test_orig = pd.read_csv("CBIS-DDSM_Data/calc_case_description_test_set.csv")

Example of an image's path    

In [182]:
calc_test_orig["image file path"][0]

'Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.85935434310203356712688695661986996009/1.3.6.1.4.1.9590.100.1.2.374115997511889073021386151921807063992/000000.dcm'

#### Match and Combine Description Data
Currently the mass and calcification description files have different names. The names are matched to combine files

In [183]:
# Columns current names and new names
col_new_names = ["breast_density", "breast_side", "image_view", "abnormality_type", "pathology", "image_path", "cropped_image_path", "ROI_mask_path"] 
mass_names = ["breast_density", "left or right breast", "image view", "abnormality type", "pathology", "image file path", "cropped image file path", "ROI mask file path"]
calc_names = ["breast density", "left or right breast", "image view", "abnormality type", "pathology", "image file path", "cropped image file path", "ROI mask file path"]

In [184]:
# Rename columns
mass_train_renamed = mass_train_orig.rename(columns={name:new_name for name, new_name in zip(mass_names, col_new_names)})
mass_test_renamed = mass_test_orig.rename(columns={name:new_name for name, new_name in zip(mass_names, col_new_names)})
calc_train_renamed = calc_train_orig.rename(columns={name:new_name for name, new_name in zip(calc_names, col_new_names)})
calc_test_renamed = calc_test_orig.rename(columns={name:new_name for name, new_name in zip(calc_names, col_new_names)})

In [185]:
# Print number of cases for file
print("Original Number of Cases:")
print(" - Mass trained cases: ", len(mass_train_renamed))
print(" - Mass trained cases: ", len(mass_test_renamed))
print(" - Calc trained cases: ", len(calc_train_renamed))
print(" - Calc trained cases: ", len(calc_test_renamed))

Original Number of Cases:
 - Mass trained cases:  1318
 - Mass trained cases:  378
 - Calc trained cases:  1546
 - Calc trained cases:  326


In [186]:
# Combine needed columns and print number of cases combining training and testing data
train_orig = []
test_orig = []
train_orig = pd.concat([mass_train_renamed[col_new_names], calc_train_renamed[col_new_names]], ignore_index=True)
test_orig = pd.concat([mass_test_renamed[col_new_names], calc_test_renamed[col_new_names]], ignore_index=True)
print("New Number of Cases per Set:")
print(" - Train cases: ", len(train_orig))
print(" - Test cases: ", len(test_orig))
print("\n", train_orig.columns)

New Number of Cases per Set:
 - Train cases:  2864
 - Test cases:  704

 Index(['breast_density', 'breast_side', 'image_view', 'abnormality_type',
       'pathology', 'image_path', 'cropped_image_path', 'ROI_mask_path'],
      dtype='object')


In [187]:
# Add a split marker column
train_orig["split"] = ["train"] * len(train_orig) 
test_orig["split"] = ["test"] * len(test_orig) 

In [188]:
print(train_orig.columns)

Index(['breast_density', 'breast_side', 'image_view', 'abnormality_type',
       'pathology', 'image_path', 'cropped_image_path', 'ROI_mask_path',
       'split'],
      dtype='object')


In [189]:
test_orig.columns == train_orig.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

#### Extract series UIDs from description paths 
The decompressed images are stored in one directory where the name is the series ID number generated using the metadata file. To connect the description with its respective image already decompresed, the series UIDs are extracted from paths from descriptions files to match with images stored in images_png 

In [190]:
# Function for extracting description series UIDs from paths stored in the description files 
def get_series_from_descriptions(im_types, new_cols, train, test, new_train, new_test):
    for im_type, col in zip(im_types, new_cols):
        train_set = [serie.split("/")[2] for serie in train[im_type]]
        test_set = [serie.split("/")[2] for serie in test[im_type]]
        new_train[col] = train_set
        new_test[col] = test_set
    return new_train, new_test

In [191]:
# Extract description series UIDs from paths stored in the description files 
train_desc, test_desc = get_series_from_descriptions(["image_path", "cropped_image_path", "ROI_mask_path"], 
                                                     ["series_uid_full", "series_uid_crop", "series_uid_roi"], 
                                                     train_orig, test_orig, train_desc, test_desc)

In [192]:
# Examples of the description of two cases. The new columns are included. 
print("Two examples of description data cases with series_uid included: \n")
print(train_desc.iloc[0], "\n")
print(train_desc.iloc[len(train_desc)-1])

Two examples of description data cases with series_uid included: 

breast_density                                                        3
breast_side                                                        LEFT
image_view                                                           CC
abnormality_type                                                   mass
pathology                                                     MALIGNANT
image_path            Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...
cropped_image_path    Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...
ROI_mask_path         Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...
split                                                             train
series_uid_full       1.3.6.1.4.1.9590.100.1.2.342386194811267636608...
series_uid_crop       1.3.6.1.4.1.9590.100.1.2.296736403313792599626...
series_uid_roi        1.3.6.1.4.1.9590.100.1.2.296736403313792599626...
Name: 0, dtype: object 

breast_density                              

#### Gets images new locations and make a new data frame 
Create a new data frame with information of each image.For each image saved: 
- extract series_uid
- find its respective information in the description (train/test) sets
- find its respective metadata. 

In [193]:
# Extracts paths from images stored in images_png directory 
path_to_imgs = "CBIS-DDSM_Clean_Data/labeled_images_png/"
images_paths = os.listdir(path_to_imgs)


print("Number of images in total:")
print(" - ", len(images_paths))

print("\nExamples of PNG images paths:")
images_paths[:2]

Number of images in total:
 -  10239

Examples of PNG images paths:


['1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132_cropped.png',
 '1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132_roi.png']

In [194]:
# creates a list of series UIDs by removing file extension and last 3 digits 
images_series = [path.split("_")[0] for path in images_paths]
images_series[:3]

['1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132',
 '1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132',
 '1.3.6.1.4.1.9590.100.1.2.100131208110604806117271735422083351547']

In [195]:
# verify the series are unique
print("Unique paths:", len(set(images_paths)), "out of ", len(images_paths))
print("Unique series:", len(set(images_series)), "out of ", len(images_series))

Unique paths: 10239 out of  10239
Unique series: 6775 out of  10239


#### Extract metadata
As we can see in the folowing cell, some series cantain more than one images. For that reason, the path can be used to diferenciate images by adding a column with the type of image.

In [196]:
# Function for matching images paths to metadata 
def get_images_metadata(images_series, metadata_series, metadata_col):
    new_col = []
    # uses description series uids
    for serie in images_series:
        # makes sure the series is in the metadata
        if serie in list(metadata_series):
            index = list(metadata_series).index(serie)
            new_col.append(metadata_col[index])
        else: 
            new_col.append(None)
            print(serie, "not found")
    # if col_type:
    #     new_col = pd.Series(new_col)
    #     new_col = new_col.replace("full mammogram images", "full")
    #     new_col = new_col.replace("ROI mask images", "ROI")
    #     new_col = new_col.replace("cropped images", "cropped")
    return new_col
            

In [197]:
# Create dataframe and add metadata
images_data = pd.DataFrame()

imges_types = [path.split("_")[1].split(".")[0] for path in images_paths]
mapping = {"full": "00", "cropped": "01", "roi": "02"}
# Adds metadata to new dataframe
images_data["image_id"] = [serie[-5:] + "_" + mapping[im_type] for serie, im_type in zip(images_series, imges_types)]
images_data["image_type"] = imges_types
images_data["image_path"] = images_paths
images_data["series_uid"] = images_series
images_data["subject_id"] = get_images_metadata(images_series, metadata_df["Series UID"], metadata_df["Subject ID"])
images_data["study_uid"] = get_images_metadata(images_series, metadata_df["Series UID"], metadata_df["Study UID"])


In [198]:
images_data.iloc[:3]

,image_id,image_type,image_path,series_uid,subject_id,study_uid
0,91132_01,cropped,1.3.6.1.4.1.9590.100.1.2.100018879311824535125...,1.3.6.1.4.1.9590.100.1.2.100018879311824535125...,Calc-Training_P_01128_RIGHT_CC_1,1.3.6.1.4.1.9590.100.1.2.346810468613236696616...
1,91132_02,roi,1.3.6.1.4.1.9590.100.1.2.100018879311824535125...,1.3.6.1.4.1.9590.100.1.2.100018879311824535125...,Calc-Training_P_01128_RIGHT_CC_1,1.3.6.1.4.1.9590.100.1.2.346810468613236696616...
2,51547_00,full,1.3.6.1.4.1.9590.100.1.2.100131208110604806117...,1.3.6.1.4.1.9590.100.1.2.100131208110604806117...,Calc-Training_P_01107_LEFT_CC,1.3.6.1.4.1.9590.100.1.2.113816182611334006337...


#### Extract metadata
The description files contain three columns to specify the series that are part of one description. However, to find the data we need to identify what series column should be used (full, roi, or cropped). Moreover, it needs to be verifyied if all matched images found per column  match their respective data.

In [199]:
images_data["image_type"][:4]

0    cropped
1        roi
2       full
3       full
Name: image_type, dtype: object

In [200]:
# Function for matching images paths to description data 
def get_images_description(images_series, image_types, descr_series_cols, descr_data):
    rows = [None] * len(images_series)
    multi_label_images = []
    not_found = []
    for i, serie, im_type in zip(range(len(images_series)), images_series, image_types):
        
        # makes sure the series is in the descriptions data 
        if im_type == "full": 
            matched_rows = descr_data[descr_series_cols[0] == serie]
        elif im_type == "cropped":
            matched_rows = descr_data[descr_series_cols[1] == serie]
        elif im_type == "roi":
            matched_rows = descr_data[descr_series_cols[2] == serie]
            
        # Makes sure the data is identical in the rows found
        if len(matched_rows) > 1:
            identical = (matched_rows == matched_rows.iloc[0]).all().all()
            if identical:
                rows[i] = list(matched_rows.iloc[0])
            else:
                rows[i] = [None] * len(matched_rows.iloc[0])
                multi_label_images.append(serie)
        elif len(matched_rows) == 1:
            rows[i] = list(matched_rows.iloc[0])
        else:
            rows[i] = [None] * 6
            not_found.append(serie)
    print("Number of cases that contain multiple labels for one image: ", len(multi_label_images))
    print("Number of series that where NOT found in the description data: ", len(not_found))
    return rows, multi_label_images, not_found
        
        



In [201]:
# Adds Description data to new dataframe
col_names = ["breast_density", "breast_side", "image_view", "abnormality_type", "pathology", "split", "series_uid_full", "series_uid_crop", "series_uid_roi"] 
all_desc_data = pd.concat([train_desc[col_names], test_desc[col_names]], ignore_index=True)
images_desc, multi_label_cases, not_found_cases = get_images_description(images_series,
                                                 images_data["image_type"],
                                                 [all_desc_data["series_uid_full"], all_desc_data["series_uid_crop"], all_desc_data["series_uid_roi"]],
                                                 all_desc_data[all_desc_data.columns[:6]] )
images_desc[0]

Number of cases that contain multiple labels for one image:  19
Number of series that where NOT found in the description data:  2


[np.int64(3), 'RIGHT', 'CC', 'calcification', 'MALIGNANT', 'train']

#### Combine metadata, paths and description data

In [203]:
# create a dataframe 
images_desc_df = pd.DataFrame(images_desc, columns=col_names[:6])
images_desc_df[:4]

,breast_density,breast_side,image_view,abnormality_type,pathology,split
0,3.0,RIGHT,CC,calcification,MALIGNANT,train
1,3.0,RIGHT,CC,calcification,MALIGNANT,train
2,2.0,LEFT,CC,calcification,BENIGN,train
3,2.0,LEFT,MLO,mass,MALIGNANT,test


In [224]:
# combine images data (files paths and metadata) with description data 
all_images_data = pd.DataFrame()
all_images_data = pd.concat([images_data, images_desc_df], axis=1)
all_images_data.iloc[4765]

image_id                                                     91565_02
image_type                                                        roi
image_path          1.3.6.1.4.1.9590.100.1.2.276112479100234745155...
series_uid          1.3.6.1.4.1.9590.100.1.2.276112479100234745155...
subject_id                           Calc-Training_P_00474_LEFT_MLO_1
study_uid           1.3.6.1.4.1.9590.100.1.2.317367128312662574506...
breast_density                                                    NaN
breast_side                                                      None
image_view                                                       None
abnormality_type                                                 None
pathology                                                        None
split                                                            None
Name: 4765, dtype: object

In [225]:
len(all_images_data)

10239

In [231]:
# remove rows that include None
all_images_data = all_images_data[all_images_data["pathology"].notna()]
len(all_images_data) 

10218

#### Separate full images, cropped and ROIs masks

In [232]:
# Separate type of images by group 
full_images = all_images_data[all_images_data["image_type"] == "full"]
cropped_images = all_images_data[all_images_data["image_type"] == "cropped"]
roi_images = all_images_data[all_images_data["image_type"] == "roi"]
full_images[:4]

,image_id,image_type,image_path,series_uid,subject_id,study_uid,breast_density,breast_side,image_view,abnormality_type,pathology,split
2,51547_00,full,1.3.6.1.4.1.9590.100.1.2.100131208110604806117...,1.3.6.1.4.1.9590.100.1.2.100131208110604806117...,Calc-Training_P_01107_LEFT_CC,1.3.6.1.4.1.9590.100.1.2.113816182611334006337...,2.0,LEFT,CC,calcification,BENIGN,train
3,62869_00,full,1.3.6.1.4.1.9590.100.1.2.100522099512256189513...,1.3.6.1.4.1.9590.100.1.2.100522099512256189513...,Mass-Test_P_00576_LEFT_MLO,1.3.6.1.4.1.9590.100.1.2.286066835611001826026...,2.0,LEFT,MLO,mass,MALIGNANT,test
4,45647_00,full,1.3.6.1.4.1.9590.100.1.2.100522676511025180541...,1.3.6.1.4.1.9590.100.1.2.100522676511025180541...,Mass-Test_P_01510_RIGHT_MLO,1.3.6.1.4.1.9590.100.1.2.403479789212123359501...,4.0,RIGHT,MLO,mass,BENIGN,test
7,99884_00,full,1.3.6.1.4.1.9590.100.1.2.100579676611077714807...,1.3.6.1.4.1.9590.100.1.2.100579676611077714807...,Calc-Training_P_00685_RIGHT_CC,1.3.6.1.4.1.9590.100.1.2.137434121012998881531...,3.0,RIGHT,CC,calcification,MALIGNANT,train


#### Split data based on split marker

In [235]:
train_full = full_images[full_images["split"] == "train"]
test_full = full_images[full_images["split"] == "test"]
total_full = (len(train_full) + len(test_full))
print("\nTraining set size:", len(train_full))
print("Testing set size:", len(test_full), "\n")
print("Total full images cases: ", total_full)
print("Train/Test ratio:", (len(train_full) / total_full), " / ", (len(test_full) / total_full))


Training set size: 2443
Testing set size: 641 

Total full images cases:  3084
Train/Test ratio: 0.7921530479896238  /  0.20784695201037615


In [236]:
print(train_full.iloc[0], "\n")
print(test_full.iloc[0])

image_id                                                     51547_00
image_type                                                       full
image_path          1.3.6.1.4.1.9590.100.1.2.100131208110604806117...
series_uid          1.3.6.1.4.1.9590.100.1.2.100131208110604806117...
subject_id                              Calc-Training_P_01107_LEFT_CC
study_uid           1.3.6.1.4.1.9590.100.1.2.113816182611334006337...
breast_density                                                    2.0
breast_side                                                      LEFT
image_view                                                         CC
abnormality_type                                        calcification
pathology                                                      BENIGN
split                                                           train
Name: 2, dtype: object 

image_id                                                     62869_00
image_type                                                       

### Save new organized data in Dataframe
Save full images data to csv files to be used in model training

In [238]:
train_full.columns

Index(['image_id', 'image_type', 'image_path', 'series_uid', 'subject_id',
       'study_uid', 'breast_density', 'breast_side', 'image_view',
       'abnormality_type', 'pathology', 'split'],
      dtype='object')

In [239]:
# save full images dta to csv files to be used in model training
train_full.to_csv("CBIS-DDSM_Clean_Data/train_full.csv", index=False)
test_full.to_csv("CBIS-DDSM_Clean_Data/test_full.csv", index=False)